## Fast Agent

First, let's set up our SQLite database connection using the Chinook sample database.


In [ ]:
import { SqlDatabase } from "@langchain/classic/sql_db";
import { DataSource } from "typeorm";

const datasource = new DataSource({
    type: "sqlite",
    database: "./Chinook.db", // Replace with the link to your database
});
const db = await SqlDatabase.fromDataSourceParams({
    appDataSource: datasource,
});

Create a tool that executes SQL queries against our database.


In [ ]:
import { tool } from "langchain";
import { z } from "zod";

export const executeSQL = tool(({ query }) => {
    return db.run(query)
}, {
    name: "execute_sql",
    description: "Execute a SQLite command and return results.",
    schema: z.object({ query: z.string() })
})

Define a system prompt with clear rules for the agent to follow when querying the database.


In [ ]:
export const systemPrompt = `You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool \`execute_sql\` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows of output unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.`

Now we can create our agent by combining the model, tools, and system prompt.


In [ ]:
import * as dotenv from "dotenv/config";
import { createAgent } from "langchain";

const agent = createAgent({
    model: "openai:gpt-5",
    tools: [executeSQL],
    systemPrompt,
})

Let's visualize the agent's graph structure.


In [ ]:
await agent.drawMermaidPng()

Time to test it! Let's ask which table has the most entries.


In [ ]:
const question = "Which table has the largest number of entries?"
const stream = await agent.stream({
    messages: question
}, {
    stream_mode: "values",
})

for await (const step of stream) {
    const lastMessage = step?.messages?.at(-1)
    const content = lastMessage.content || `Tool calls: ${JSON.stringify(lastMessage.tool_calls.map(t => `${t.name}: ${JSON.stringify(t.args)}`).join(", "))}`;
    console.log(`${lastMessage.type} - ${content}`)
}

Now let's try a more complex analytical query involving joins and aggregations.


In [ ]:
const question = "Which genre on average has the longest tracks?"
const stream = await agent.stream({
    messages: question
}, {
    stream_mode: "values",
})

for await (const step of stream) {
    const lastMessage = step?.messages?.at(-1)
    const content = lastMessage.content || `Tool calls: ${JSON.stringify(lastMessage.tool_calls.map(t => `${t.name}: ${JSON.stringify(t.args)}`).join(", "))}`;
    console.log(`${lastMessage.type} - ${content}`)
}

Let's ask for a simple list of all tables in the database.


In [ ]:
const question = "What are all the tables?";
const stream = await agent.stream({
    messages: question
}, {
    stream_mode: "values",
})

for await (const step of stream) {
    const lastMessage = step?.messages?.at(-1)
    const content = lastMessage.content || `Tool calls: ${JSON.stringify(lastMessage.tool_calls.map(t => `${t.name}: ${JSON.stringify(t.args)}`).join(", "))}`;
    console.log(`${lastMessage.type} - ${content}`)
}